In [ ]:
%%HTML
<style type="text/css">

div.h1 {
    font-size: 32px; 
    margin-bottom:2px;
    background-color: steelblue; 
    color: white; 
    text-align: center;
}
div.h2 {
    background-color: steelblue; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 24px; 
    max-width: 1500px; 
    margin-top: 50px;
    margin-bottom:4px;
    
}
div.h3 {
    color: steelblue; 
    font-size: 20px; 
    margin-top: 4px; 
    margin-bottom:8px;
}
div.h4 {
    font-size: 15px; 
    margin-top: 20px; 
    margin-bottom: 8px;
}

</style>


<div class="h1">30 Days of ML- Competition</div>



This notebook is created as part  of 30 day ML challenge. This book is divided in 3 part EDA & kfold(https://www.kaggle.com/yogidsba/eda-kfold-30-day-ml) , Modelbuilding using XGBoost ,Understanding blending

**About the data**
The dataset is used for this competition is synthetic, but based on a real dataset. The original dataset deals with predicting the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.


**Evaluation**

Submissions are scored on the root mean squared error.


**Special mention**
I learned about Blending from Abhishek Thakur YoutubeChannel. For that code you can refer his notebook or watch youtube channel.(https://www.youtube.com/watch?v=ISZYWtvoCAc)Thank you @Abhisek

<div class="h1">Table of Contents</div>

  - <a href='#il'>Import Libraries</a>
  - <a href='#rd'>Read and Understand Data</a>
  - <a href='#mb'>Model Building </a>
  - <a href='#bl'>Blending</a> 

**Blending**

Blending combine the decisions from multiple models to improve the overall performance. This approach allows for better predictive performance compared to a single model.It help us to improve performance and increase accuracy.  

Blending ensemble are a type of stacking where the meta-model is fit using predictions on a holdout validation dataset instead of out-of-fold predictions.

For example, a linear regression model when predicting a numerical value or a logistic regression model when predicting a class label would calculate a weighted sum of the predictions made by base models and would be considered a blending of predictions.

 Understanding the blending process
 - The train set is split into two parts, viz-training and validation sets.
 - Model(s) are fit on the training set.
 - The predictions are made on the validation set and the test set.
 - The validation set and its predictions are used as features to build a new model.
 - This model is used to make final predictions on the test and meta-features.
 
**Resources**

https://machinelearningmastery.com/blending-ensemble-machine-learning-with-python/

https://www.youtube.com/watch?v=ISZYWtvoCAc ( Abhishek Thakur Video)

https://www.youtube.com/watch?v=TuIgtitqJho ( Abhishek Thakur Video)

https://mlwave.com/kaggle-ensembling-guide/?lipi=urn%3Ali%3Apage%3Ad_flagship3_pulse_read%3BPZ4T3JLHTu%2BOWNI0d5kFbg%3D%3D





<a id='il'></a>
<div class="h3">Import Libraries</div>

In [ ]:
# Familiar imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 # For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
print("loaded libraries")

<a id='rd'></a>
<div class="h3">Read and Understand Data</div>



In [ ]:
# Load the training data
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv")
# Preview the data
df.head()

<a id='mb'></a>
<div class="h3">Model Building</div>


In [ ]:

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

**1st Model XGBoost**

In [ ]:
final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    params ={'random_state': 1,   
             'booster': 'gbtree',
             'n_estimators': 10000,
             'learning_rate': 0.03628302216953097,
             'reg_lambda': 0.0008746338866473539,
             'reg_alpha': 23.13181079976304,
             'subsample': 0.7875490025178415,
             'colsample_bytree': 0.11807135201147481,
             'max_depth': 3    } 
    model = XGBRegressor(n_jobs=4,**params)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
#create a newdataframe  with id and the predications from validation set
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("train_pred_1.csv", index=False)
#create a new test csv with the final test prediction
sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("test_pred_1.csv", index=False)



**2nd model XGboost different parameters**

In [ ]:

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    params = {'learning_rate': 0.07853392035787837,
              'reg_lambda': 1.7549293092194938e-05,
              'reg_alpha': 14.68267919457715,
              'subsample': 0.8031450486786944,
              'colsample_bytree': 0.170759104940733,
              'max_depth': 3}
    model = XGBRegressor(random_state=fold,n_jobs=4,n_estimators=5000,**params)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("test_pred_2.csv", index=False)

**3rd Model CatBoost**

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    params = {'learning_rate': 0.02,
              'grow_policy':'Depthwise',
              'iterations':4000,
              'use_best_model':True,
              'eval_metric':'RMSE',
              'od_type':'iter',
              'od_wait':40,
              'max_depth': 3 }
       
    model = CatBoostRegressor(
        random_state=fold,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=400, eval_set=[(xvalid, yvalid)], verbose=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("test_pred_3.csv", index=False)

<a id='bl'></a>
<div class="h3">Blending Models</div>


Blending 3 different models

In [ ]:
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("train_pred_1.csv")
df2 = pd.read_csv("train_pred_2.csv")
df3 = pd.read_csv("train_pred_3.csv")

df_test1 = pd.read_csv("test_pred_1.csv")
df_test2 = pd.read_csv("test_pred_2.csv")
df_test3 = pd.read_csv("test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df.head()


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
useful_features = ["pred_1", "pred_2","pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LinearRegression()
    
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))


In [ ]:
model.coef_

In [ ]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("Blended.csv", index=False)

